# Provide source and destination ML- label name 

In [66]:
# pip install jira
from jira import JIRA
import getpass
import pandas as pd
import numpy as np
import os.path
from pandas import ExcelWriter
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.utils.fixes import signature
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
import seaborn as sns
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle


#input( "Username: ")
userName = 'bob.ansari@intelex.com' 
password = getpass.getpass(prompt='Username: '+userName+'\r\nPassword: ')

auth_jira = JIRA(server = 'https://intelex.atlassian.net', auth=( userName, password ))

print("Logged in")

Username: bob.ansari@intelex.com
Password: ········
Logged in


In [78]:
def get_ml_labels(changing_label):
    all_issues = auth_jira.search_issues('"Tech Team" = "Team 1" AND Sprint is not EMPTY AND "Story Points" is not EMPTY and labels is not EMPTY order by priority desc', maxResults=1000)
    label_list = []
    i = 0
    #if item in my_list:
    for issue in all_issues:
        for label in issue.fields.labels:
            if 'ML-' in label and label not in label_list:
                if label == changing_label:
                    labels_df.loc[i] = ["*** "+label+" ***"]
                else:
                    labels_df.loc[i] = [label]
                i = i +1
                label_list.append(label)

    return labels_df

In [80]:
get_ml_labels("")

,Labels
0,ML-M-Code-Business
1,ML-M-Configuration
2,ML-M-Test-Manual
3,ML-M-Communication
4,ML-S-Code-Business
5,ML-M-Test-Automation
6,ML-S-Code-UI
7,ML-M-Code-Script
8,ML-M-Studying
9,ML-L-Code-Business


In [68]:
srcMlName = input( "Source ML: ")
destMlName = input( "Destination ML: ")
src_label_issues = auth_jira.search_issues('labels = "'+srcMlName+'" order by priority desc', maxResults=1000)
  
labels_df = pd.DataFrame(columns=['Labels'])


Source ML: 1
Destination ML: 2


In [69]:
get_ml_labels(srcMlName)


,Labels
0,ML-M-Code-Business
1,ML-M-Configuration
2,ML-M-Test-Manual
3,ML-M-Communication
4,ML-S-Code-Business
5,ML-M-Test-Automation
6,ML-S-Code-UI
7,ML-M-Code-Script
8,ML-M-Studying
9,ML-L-Code-Business


In [61]:
def change_labels(from_src, to_dest):
    print("Changing label from  \"" + from_src + "\" to \""+to_dest+"\" for the following issues:")
    for issue in src_label_issues:
        print("\t- Issue #"+ issue.key)
        issue.fields.labels.append(to_dest)
        issue.fields.labels.remove(from_src)
        issue.update(fields={"labels": issue.fields.labels})
        print("\t  Done!")

In [62]:
change_labels(srcMlName, destMlName)
get_ml_labels(destMlName)

Changing label from  "ML-Code-Alliance-Automation" to "ML-M-Code-Automation" for the following issues:
	- Issue #ILX-8399
	  Done!
	- Issue #ILX-7221
	  Done!


,Labels
0,ML-M-Code-Business
1,ML-M-Configuration
2,ML-M-Test-Manual
3,ML-M-Communication
4,ML-S-Code-Business
5,ML-M-Test-Automation
6,ML-S-Code-UI
7,ML-M-Code-Script
8,ML-M-Studying
9,ML-L-Code-Business


In [63]:
get_ml_labels("")

,Labels
0,ML-M-Code-Business
1,ML-M-Configuration
2,ML-M-Test-Manual
3,ML-M-Communication
4,ML-S-Code-Business
5,ML-M-Test-Automation
6,ML-S-Code-UI
7,ML-M-Code-Script
8,ML-M-Studying
9,ML-L-Code-Business
